In [13]:
import os
from rdflib import ConjunctiveGraph, Namespace
import json
import requests

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
# g.load(os.environ.get('EDAM_PATH', '../EDAM_dev.owl'), format='xml')
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")

Loading graph ...done!


In [14]:
# Compile SPARQL query
query_term = """
SELECT DISTINCT ?edam_class ?seealso WHERE {
    ?edam_class a owl:Class .
    ?edam_class rdfs:seeAlso ?seealso .
    FILTER (regex(str(?seealso), "wikipedia")) .
} 
"""
# Declare hash tables for results
ids = {}
terms = {}
errs = {}

# Run SPARQL query and collate results
errfound = False    
report = list()
results = g.query(query_term)

Analyse results of query.

In [15]:
wikipedia_urls = []

for r in results :
    edam_uri      = str(r['edam_class'])
    seealso = str(r['seealso']) 
    wikipedia_urls.append(seealso)

In [30]:
def query_wikidata(uri):
    results = []
    endpoint = "https://query.wikidata.org/sparql"
    query = """
SELECT * WHERE {
    <"""+uri+"""> schema:about ?wd_entity
}
        """

    h = {"Accept": "application/sparql-results+json"}
    p = {"query": query}

    res = requests.get(endpoint, headers=h, params=p, verify=False)
    
    for r in res.json()['results']['bindings']:
        results.append(r['wd_entity']['value'])
    return results

In [33]:
for l in wikipedia_urls[:9]:
    wididata = query_wikidata(l)
    print(f'{l} -> {wididata}')

/Users/gaignard-a/miniconda3/envs/edamverify/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/gaignard-a/miniconda3/envs/edamverify/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://en.wikipedia.org/wiki/Subcellular_localization -> ['http://www.wikidata.org/entity/Q7630919']
http://en.wikipedia.org/wiki/Systems_biology -> []


/Users/gaignard-a/miniconda3/envs/edamverify/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://en.wikipedia.org/wiki/Toxicology -> ['http://www.wikidata.org/entity/Q7218']


/Users/gaignard-a/miniconda3/envs/edamverify/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


...

https://en.wikipedia.org/wiki/Epigenetics -> ['http://www.wikidata.org/entity/Q26939']


/Users/gaignard-a/miniconda3/envs/edamverify/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://en.wikipedia.org/wiki/Computational_immunology -> ['http://www.wikidata.org/entity/Q5157321']


/Users/gaignard-a/miniconda3/envs/edamverify/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://en.wikipedia.org/wiki/Drug_development -> ['http://www.wikidata.org/entity/Q2198549']
